In [ ]:
# Instalar librería para generar Word si no existe
!pip install python-docx

# MEMORIA DE CÁLCULO DE INGENIERÍA: ROBOT SCARA RRP
## ANÁLISIS MULTIDISCIPLINARIO Y GENERACIÓN DE REPORTE

**Proyecto Final de Ingeniería** 
**Instrucciones:** Ejecuta todas las celdas ('Run All'). Al finalizar, este notebook generará automáticamente un archivo Word (.docx) con todos los resultados, tablas y gráficas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os

# Configuración Gráfica
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

In [ ]:
# === CONSTANTES FÍSICAS (INPUTS) ===
L1 = 0.29; L2 = 0.125; D1 = 0.136; Z_max = 0.055
m1 = 0.65; m2 = 0.25; m_load = 0.125; g = 9.81
V_in = 7.4; Tau_stall = 70.0; Km = 8.2; I_idle = 0.3
Mu_roll = 0.002; R_bearing = 0.04
Sigma_Y = 50e6

print("Datos cargados. Preparando simulaciones...")

In [ ]:
# --- MATRICES DH ---
def dh(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])
T_final = dh(0, D1, L1, 0) @ dh(0, 0, L2, np.pi)

# --- PLOT 3D WORKSPACE ---
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
th1, th2 = np.meshgrid(np.linspace(-1.5, 1.5, 20), np.linspace(-2.5, 2.5, 20))
X = L1*np.cos(th1) + L2*np.cos(th1+th2)
Y = L1*np.sin(th1) + L2*np.sin(th1+th2)
Z = np.zeros_like(X)
ax.plot_surface(X, Y, Z, color='cyan', alpha=0.3)
ax.plot_surface(X, Y, Z+Z_max, color='blue', alpha=0.3)
ax.set_title('Espacio de Trabajo SCARA')
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')

# GUARDAR IMAGEN PARA EL WORD
plt.savefig('img_workspace.png', dpi=150)
plt.show()

In [ ]:
t = np.linspace(0, 1.5, 100)
tau = t/t[-1]
s = 10*tau**3 - 15*tau**4 + 6*tau**5 # Polinomio 5to orden
q = (np.pi/2) * s
vel = np.gradient(q, t)
acc = np.gradient(vel, t)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(t, np.degrees(q), 'k', label='Pos')
ax1.set_ylabel('Posición (°)')
ax1.set_title('Perfil Cinemático (Jerk Control)')
ax1.grid(True)
ax2.plot(t, acc, 'r', label='Acc')
ax2.set_ylabel('Acel (rad/s²)')
ax2.set_xlabel('Tiempo (s)')
ax2.grid(True)

plt.savefig('img_control.png', dpi=150)
plt.show()

In [ ]:
# Inercia
I_tot = (1/3)*m1*L1**2 + (1/3)*m2*L2**2 + m2*L1**2 + m_load*(L1+L2)**2
# Torque
N_ax = (m1+m2+m_load)*g
Tau_fric = Mu_roll * N_ax * R_bearing
Tau_dyn = I_tot * acc + Tau_fric
Tau_kgcm = Tau_dyn * 10.197
# Corriente
I_mot = np.abs(Tau_kgcm/Km) + I_idle

fig, ax1 = plt.subplots()
ax1.plot(t, Tau_kgcm, 'purple')
ax1.set_ylabel('Torque (kg-cm)', color='purple')
ax1.axhline(Tau_stall, color='g', ls='--')
ax2 = ax1.twinx()
ax2.plot(t, I_mot, 'orange')
ax2.set_ylabel('Corriente (A)', color='orange')
plt.title('Dinámica y Consumo')

plt.savefig('img_dinamica.png', dpi=150)
plt.show()

In [ ]:
x = np.linspace(0, L1, 100)
M_x = -((m2+m_load)*g*(L1-x)) - ((m1*g/L1)*(L1-x)**2)/2
M_max = np.max(np.abs(M_x))
sigma = (M_max * 0.02) / ((0.02*0.04**3)/12)
FS = Sigma_Y / sigma

plt.figure()
plt.fill_between(x*100, np.abs(M_x), color='red', alpha=0.3)
plt.plot(x*100, np.abs(M_x), 'r')
plt.title(f'Momento Flector (FS = {FS:.1f})')
plt.xlabel('Longitud (cm)'); plt.ylabel('Momento (Nm)')

plt.savefig('img_materiales.png', dpi=150)
plt.show()

In [ ]:
v = np.linspace(0, 5, 100)
mu = Mu_roll + (0.5 - Mu_roll)*np.exp(-v*2)
plt.figure()
plt.plot(v, mu, 'g')
plt.title('Curva Stribeck (Eficiencia Rodamiento)')
plt.xlabel('Velocidad'); plt.ylabel('Coeficiente Fricción')

plt.savefig('img_stribeck.png', dpi=150)
plt.show()

## 🚀 GENERACIÓN AUTOMÁTICA DEL REPORTE (.docx)
Esta celda recopila todas las variables calculadas y las imágenes guardadas para construir el documento final.

In [ ]:
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Crear Documento
doc = Document()
style = doc.styles['Normal']
style.font.name = 'Times New Roman'
style.font.size = Pt(11)

def add_h1(text):
    h = doc.add_heading(text, 1)
    h.style.font.color.rgb = RGBColor(0, 0, 0)
    h.alignment = WD_ALIGN_PARAGRAPH.CENTER

def add_kv(k, v):
    p = doc.add_paragraph()
    p.add_run(f"• {k}: ").bold = True
    p.add_run(str(v))

# --- PORTADA ---
doc.add_heading('MEMORIA DE CÁLCULO DE INGENIERÍA', 0).alignment = WD_ALIGN_PARAGRAPH.CENTER
doc.add_paragraph('ROBOT SCARA RRP - PROYECTO FINAL').alignment = WD_ALIGN_PARAGRAPH.CENTER
doc.add_page_break()

# --- CAP 1: PARAMETROS ---
add_h1("1. PARÁMETROS DE DISEÑO")
doc.add_paragraph("Datos extraídos del prototipo físico:")
table = doc.add_table(rows=1, cols=2)
table.style = 'Light Grid Accent 1'
hdr = table.rows[0].cells; hdr[0].text = 'Parámetro'; hdr[1].text = 'Valor'
params = [
    ('Longitud L1', f'{L1} m'), ('Longitud L2', f'{L2} m'),
    ('Masa Brazo 1', f'{m1} kg'), ('Masa Brazo 2', f'{m2} kg'),
    ('Carga Útil', f'{m_load} kg'), ('Torque Motor', f'{Tau_stall} kg-cm')
]
for k,v in params:
    row = table.add_row().cells
    row[0].text = k; row[1].text = v

# --- CAP 2: CINEMATICA ---
add_h1("2. CINEMÁTICA Y MATRICES")
doc.add_paragraph("Matriz de Transformación Final T_02 (Numérica):")
doc.add_paragraph(str(np.round(T_final, 3))).style = 'Quote'
doc.add_paragraph("Visualización del volumen de trabajo accesible:")
doc.add_picture('img_workspace.png', width=Inches(5))

# --- CAP 3: CONTROL ---
add_h1("3. TEORÍA DE CONTROL")
doc.add_paragraph("Se implementó un generador de trayectorias de 5to orden para minimizar el Jerk.")
doc.add_picture('img_control.png', width=Inches(5))

# --- CAP 4: DINAMICA ---
add_h1("4. DINÁMICA Y ELÉCTRICA")
doc.add_paragraph("Resultados de la simulación multifísica:")
add_kv("Inercia Total Calculada", f"{I_tot:.4f} kg·m²")
add_kv("Torque Máximo Requerido", f"{np.max(Tau_kgcm):.2f} kg-cm")
add_kv("Corriente Pico Estimada", f"{np.max(I_mot):.2f} A")
doc.add_picture('img_dinamica.png', width=Inches(5))

# --- CAP 5: MATERIALES ---
add_h1("5. RESISTENCIA DE MATERIALES")
doc.add_paragraph("Análisis de viga en voladizo (PLA):")
add_kv("Momento Flector Máx", f"{M_max:.2f} Nm")
add_kv("Esfuerzo Von Mises", f"{sigma/1e6:.2f} MPa")
add_kv("FACTOR DE SEGURIDAD", f"{FS:.2f}")
doc.add_picture('img_materiales.png', width=Inches(5))

# --- CAP 6: TRIBOLOGIA ---
add_h1("6. TRIBOLOGÍA")
doc.add_paragraph("La curva de Stribeck valida el régimen de fricción por rodadura del nuevo soporte axial.")
doc.add_picture('img_stribeck.png', width=Inches(5))

# --- CAP 7: CONCLUSIONES ---
add_h1("CONCLUSIONES")
doc.add_paragraph("El diseño cumple con todos los criterios de ingeniería. El factor de seguridad estructural es >100 y el margen de torque es superior al 800%, garantizando una operación robusta y duradera.")

# GUARDAR
filename = 'Reporte_Ingenieria_SCARA.docx'
doc.save(filename)
print(f"✅ ¡ÉXITO! Documento generado: {filename}")